In [1]:
pip install pandas openpyxl


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd

print("pandas version:", pd.__version__)


pandas version: 2.2.3


In [6]:
import pandas as pd

# Load your uploaded file
file_path = "SMT_2020_Model_Data_-_LVHM.xlsx"

# Load the Excel file
xls = pd.ExcelFile(file_path)

# Show all available sheet names
print("Available sheets:", xls.sheet_names)



Available sheets: ['Toolgroups', 'PM', 'Breakdown', 'Setups', 'Transport', 'Lotrelease', 'Lotrelease - variable due dates', 'Route_Product_1', 'Route_Product_2', 'Route_Product_3', 'Route_Product_4', 'Route_Product_5', 'Route_Product_6', 'Route_Product_7', 'Route_Product_8', 'Route_Product_9', 'Route_Product_10']


In [9]:
# Load route for Product 1
route_df = pd.read_excel(file_path, sheet_name='Route_Product_1')
tools_df = pd.read_excel(file_path, sheet_name='Toolgroups')

# Preview both
print("First few route steps for Product 1:")
print(route_df.head())

print("\nTool group definitions:")
print(tools_df.head())


First few route steps for Product 1:
             ROUTE  STEP STEP DESCRIPTION       AREA         TOOLGROUP  \
0  Route_Product_1     1    004_Diffusion  Diffusion  Diffusion_FE_125   
1  Route_Product_1     2     005_Wet_Etch   Wet_Etch          WE_FE_84   
2  Route_Product_1     3      006_Def_Met    Def_Met     DefMEt_FE_118   
3  Route_Product_1     4       007_TF_Met     TF_Met      TF_Met_FE_45   
4  Route_Product_1     5    008_Diffusion  Diffusion  Diffusion_FE_127   

  PROCESSING UNIT PROCESSINGTIME DISTRIBUTION     MEAN  OFFSET PT UNITS  ...  \
0           Batch                     uniform  440.400   22.02      min  ...   
1           Wafer                     uniform    1.140    0.06      min  ...   
2             Lot                     uniform   29.880    1.49      min  ...   
3             Lot                     uniform    8.514    0.43      min  ...   
4           Batch                     uniform  437.580   21.88      min  ...   

   OFFSET.1 ST UNITS  STEP FOR LTL DE

In [10]:
import simpy

# Extract route steps from Excel
steps = route_df[['STEP', 'TOOLGROUP', 'MEAN', 'PROCESSING UNIT']].dropna()

# Convert MEAN time from string/float to float (if needed)
steps['MEAN'] = pd.to_numeric(steps['MEAN'], errors='coerce')

# SimPy environment
env = simpy.Environment()

# Create one resource (tool) per toolgroup, assume capacity=1 for now
tool_resources = {}

for tool in steps['TOOLGROUP'].unique():
    tool_resources[tool] = simpy.Resource(env, capacity=1)

# Define a lot processing function
def process_lot_through_route(env, steps, lot_id="Lot-001"):
    for index, row in steps.iterrows():
        tool_name = row['TOOLGROUP']
        proc_time = row['MEAN']  # in minutes
        proc_unit = row['PROCESSING UNIT']
        
        # Convert time to seconds for simulation if desired (optional)
        duration = proc_time  # keeping in minutes

        with tool_resources[tool_name].request() as request:
            yield request
            print(f"[{env.now:.1f} min] {lot_id} starts at {tool_name} (Step {row['STEP']})")
            yield env.timeout(duration)
            print(f"[{env.now:.1f} min] {lot_id} ends at {tool_name}")

# Launch the process
env.process(process_lot_through_route(env, steps))
env.run()


[0.0 min] Lot-001 starts at Diffusion_FE_125 (Step 1)
[440.4 min] Lot-001 ends at Diffusion_FE_125
[440.4 min] Lot-001 starts at WE_FE_84 (Step 2)
[441.5 min] Lot-001 ends at WE_FE_84
[441.5 min] Lot-001 starts at DefMEt_FE_118 (Step 3)
[471.4 min] Lot-001 ends at DefMEt_FE_118
[471.4 min] Lot-001 starts at TF_Met_FE_45 (Step 4)
[479.9 min] Lot-001 ends at TF_Met_FE_45
[479.9 min] Lot-001 starts at Diffusion_FE_127 (Step 5)
[917.5 min] Lot-001 ends at Diffusion_FE_127
[917.5 min] Lot-001 starts at DefMEt_FE_118 (Step 6)
[941.2 min] Lot-001 ends at DefMEt_FE_118
[941.2 min] Lot-001 starts at TF_Met_FE_45 (Step 7)
[951.0 min] Lot-001 ends at TF_Met_FE_45
[951.0 min] Lot-001 starts at WE_FE_108 (Step 8)
[952.0 min] Lot-001 ends at WE_FE_108
[952.0 min] Lot-001 starts at WE_FE_83 (Step 9)
[953.0 min] Lot-001 ends at WE_FE_83
[953.0 min] Lot-001 starts at WE_FE_84 (Step 10)
[953.9 min] Lot-001 ends at WE_FE_84
[953.9 min] Lot-001 starts at LithoTrack_FE_115 (Step 11)
[956.1 min] Lot-001 end